# Classificação de texto com o TensorFlow Hub: resenhas de filmes

Este estudo classifica as resenhas de filmes como *positivas* ou *negativas* usando o texto de resenha. Veremos aqui um projeto de classificação **binária** (ou de duas classes), um tipo de classificação muito usada e importante para o aprendizado de máquina.

O estudo demonstra a aplicação básica do aprendizado de transferencia com o [TensorFlow Hub](https://tfhub.dev/) e o [Keras](https://keras.io/).

O conjunto de dados usado é o do [IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb?hl=pt-br) que possui 50 mil resenhas de filmes em forma de texto. Os dados estão divididos em 25 mil para o treino e 25 mil para o teste. Esse conjunto está balancenado, isto é, há a mesma quantidade de avaliações positivas e negativas.

Esse estudo usa o [tf.keras](https://www.tensorflow.org/guide/keras?hl=pt-br), uma API de alto nível usada para criar e modelar os modelos no [TensorFlow](https://www.tensorflow.org/) e o [tensorflow_hub](https://www.tensorflow.org/hub?hl=pt-br), uma biblioteca para carregar modelos treinados do [TFHub](https://tfhub.dev/) em uma única linha de código.

In [ ]:
# Importar as bibliotecas
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Mostrar algumas informações
print(f'Versão: {tf.__version__}')
print(f'Modo eager: {tf.executing_eagerly()}')
print(f'Versão Hub: {hub.__version__}')
print('GPU está', 'disponível' if tf.config.list_physical_devices('GPU"') else 'NÃO DISPONÍVEL')

Versão: 2.8.2
Modo eager: True
Versão Hub: 0.12.0
GPU está NÃO DISPONÍVEL


## Baixar o conjunto de dados do IMDB

O conjunto de dados do IMDB está disponível em [imdb_reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews?hl=pt-br) ou nos conjuntos de dados do [TF](https://www.tensorflow.org/datasets?hl=pt-br). Esse estudo baixa o conjunto de dados para a máquina usada no Google Colab.

In [ ]:
# Vamos separar os dados de treino em 60% (15 mil dados) e para validação em 40% (10 mil dados)
# E para o teste, serão 25 mil dados
dados_treino, dados_validacao, dados_teste = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteK0DLYA/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteK0DLYA/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Explorar os dados

Vamos agora entender como os dados estão dispostos. Cada exemplo é uma frase que representa a crítica do filme e um rótulo correspondente. A sentença não é pré-processada, vem na forma bruta. O rótulo é um inteiro, sendo 0 para uma avaliação negativa e 1 para uma avaliação positiva.

Vejamos os 10 primeiros exemplos:

In [ ]:
# Mostrar como estão dispostos os dados no conjunto
lote_exemplos_treino, lote_rotulos_treino = next(iter(dados_treino.batch(10)))
lote_exemplos_treino

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Vejamos também os 10 primeiros rótulos:

In [ ]:
# Mostrar os rótulos
lote_rotulos_treino

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Construir o modelo

A rede neural é criada empilhando camadas, logo é necessário três decisões arquiteturais principais:

1. Como representar o texto?

2. Quantas camadas usar no modelo?

3. Quantas *hidden units* usar para cada camada?

Neste estudo, os dados de entrada consistem em frases. Os rótulos a serem previstos são 0 ou 1.

Um jeito de representar o texto é converter frases em vetores de incorporação. Usar uma incorporação de texto pré-treinada como a primeira camada, onde terá três vantagens:

1. Não precisamos nos preocupar com o pré-processamento de texto.

2. Seremos benificiados com o aprendizado de transferência.

3. A incorporação possui um tamanho fixo, por isso é mais simples de processar.

Para este estudo, podemos usar um **modelo de incorporação de texto pré-treinado** do [TFHub](https://tfhub.dev/) chamado [google/nnml-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2).

Há outros *embeddingds* de texto pré-treinados do TFHub que poderão ser usados neste tutorial:

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) é treinado com a mesma arquitetura NNLM nos mesmos dados que [google/nnml-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2), porém com uma dimensão de incorporação maior. Incorporações dimensionais maiores podem melhorar sua tarefa, mas pode levar mais tempo para treinar o nosso modelo.

- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) é treinado com a mesma arquitetura NNLM nos mesmos dados que [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2), porém com normalização de texto adicional, como remover pontuação. Isso pode ajudar se o texto sado contiver caracteres ou pontuação adicionais.

- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) é um modelo bem maior que produz 512 *embeddings* dimensionais treinados com um codificador de rede de média profunda (DAN).

Primeiramente, vamos criar uma camada Keras que usa o modelo do TFHub para incorporar as frases e experimentá-lo em alguns exemplos de entrada. Note qe não importa o tamanho do texto de entrada,a forma de saída dos *embeddings* é: `(num_exemplos, dimensao_embedding)`:

In [ ]:
# Usar o embedding
embedding = 'https://tfhub.dev/google/nnlm-en-dim50/2'
camada_hub = hub.KerasLayer(embedding,
                            input_shape=[],
                            dtype=tf.string,
                            trainable=True)
camada_hub(lote_exemplos_treino[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

Agora vamos construir o modelo completo:

In [ ]:
# Construção do modelo
modelo = tf.keras.Sequential()
modelo.add(camada_hub)
modelo.add(tf.keras.layers.Dense(16, activation='relu'))
modelo.add(tf.keras.layers.Dense(1))

# Mostrar o sumário do modelo
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


As camadas são empilhadas sequencialmente para construir o modelo:

1. a primeira camada é uma camada do TFHub. Essa camada utiliza um modelo salvo pré-treinado para mapear uma frase em seu vetor de incorporação. O modelo de incorporação de texto pré-treinado que estamos usando separa a frase em *tokens*, incorpora cada *token* e, em seguida, combina à incorporação. As dimensões resultantes são: `(num_exemplos, dimensao_embedding)`. Para o modelo NNLM em questão, o `dimensao_embedding` é 50.

2. O vetor de saída de comprimento fixo é canalizado através de uma camada totalmente conctada (**Dense**) com 16 *hidden units*.

3. A última camada é densamente conectada com um único neurônio de saída (é ele que nos dará a resposta).

Vamos compilar o modelo.

### Função de perda e otimizador

Um modelo necessita de uma função de perda e um otimizador para o treinamento. Como o problema que temos é de classificação binária e gera logits (uma camanda de unidade única com uma ativação linear), usaremos a função de perda `binary_crossentropy`.

Há outras opções para a função de perda, como o `mean_squared_error`. Todavia, geralmente o `binary_crossentropy` é melhor para trabalhar com probabilidades, mede a "distância" entre distribuições de probabilidade ou, em nosso caso, entre a distribuição de verdade e as previsões.

Vamos configurar o modelo para usar um otimizador e uma função de perda:

In [8]:
# Compilar o modelo
modelo.compile(optimizer='adam',
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               metrics=['accuracy'])

## Treinar o modelo

Vamos treinar o modelo por 10 épocas em mini-lotes de 512 amostras. Serão realizadas 10 iterações sobre todas as amostras nos tensores de treino. Durante o treino, monitoraremos a perda e a precisão do modelo nas 10 mil amostras do conjunto de validação:

In [10]:
# Treinar o modelo
historico = modelo.fit(dados_treino.shuffle(10**4).batch(512),
                       epochs=10,
                       validation_data=dados_validacao.batch(512),
                       verbose=1)

Epoch 1/10
30/30 [==============================] - 9s 298ms/step - loss: 0.5874 - accuracy: 0.6221 - val_loss: 0.5359 - val_accuracy: 0.7037
Epoch 2/10
30/30 [==============================] - 9s 298ms/step - loss: 0.4524 - accuracy: 0.7845 - val_loss: 0.4328 - val_accuracy: 0.7904
Epoch 3/10
30/30 [==============================] - 9s 296ms/step - loss: 0.3321 - accuracy: 0.8645 - val_loss: 0.3635 - val_accuracy: 0.8381
Epoch 4/10
30/30 [==============================] - 9s 297ms/step - loss: 0.2428 - accuracy: 0.9111 - val_loss: 0.3283 - val_accuracy: 0.8601
Epoch 5/10
30/30 [==============================] - 9s 298ms/step - loss: 0.1787 - accuracy: 0.9427 - val_loss: 0.3143 - val_accuracy: 0.8698
Epoch 6/10
30/30 [==============================] - 9s 301ms/step - loss: 0.1316 - accuracy: 0.9620 - val_loss: 0.3068 - val_accuracy: 0.8696
Epoch 7/10
30/30 [==============================] - 9s 297ms/step - loss: 0.0959 - accuracy: 0.9757 - val_loss: 0.3091 - val_accuracy: 0.8720
Epoch 

## Avaliar o modelo

Vejamos agora como o modelo se comporta. Dois valores serão retornados: a perda (número que representa o erro, quanto mais baixo melhor) e precisão (número que representa o acerto, quanto mais alto melhor):

In [12]:
# Mostrar os resultados do modelo
resultados = modelo.evaluate(dados_teste.batch(512), verbose=2)
for nome, valor in zip(modelo.metrics_names, resultados):
  print(f'{nome}: {valor:.3f}')

49/49 - 3s - loss: 0.3685 - accuracy: 0.8508 - 3s/epoch - 55ms/step
loss: 0.369
accuracy: 0.851


Melhorando o modelo, podemos chegar a uma precisão de 95%